In [1]:
# setup
import pandas as pd
import numpy as np
from CalcBenchHandler import CalcBenchHandler as CBH

%load_ext autoreload
%autoreload 2

In [158]:
# load bond trades file and get relevant columns
bond_px_path = 'data/bonds/clean_bond_close_pxs.csv'
df_pxs = pd.read_csv(bond_px_path)

dfcusip = df_pxs[['cusip_id', 'bond_sym_id', 'issuer_nm']]
dfcusip = dfcusip.drop_duplicates(subset=['cusip_id'])

In [159]:
# load calcbench list of companies
corp_path = 'data/ciks/companies.csv'
dfcorps = pd.read_csv(corp_path)

# cvt to dates
dfcorps.most_recent_filing = pd.to_datetime(dfcorps.most_recent_filing)
dfcorps.first_filing = pd.to_datetime(dfcorps.first_filing)

# filter most recent filing after 2009-12-31 and first filing before 2018-01-01
dfcorps = dfcorps[(dfcorps.most_recent_filing > '2009-12-31') &(dfcorps.first_filing <= '2018-01-01')]

# at least 2 years of filings available and full year end available
dfcorps['filing_days'] = dfcorps.most_recent_filing-dfcorps.first_filing
dfcorps = dfcorps[(dfcorps.filing_days.dt.days >= 730)]
dfcorps = dfcorps.dropna(subset=['most_recent_full_year_end'])

# drop filing date colums
dfcorps_cols = ['ticker', 'entity_name', 'entity_id', 'entity_code',
                'sic_code', 'naics', 'cik_code', 'naics_code',
                'SICGroupMinorGroupTitle']
dfcorps = dfcorps[dfcorps_cols]

# drop non-focus sics
bad_sics = r'glass|dealers|pulp|loan|concrete|gypsum|securities|optic|foundr|ferrous|dental|x-ray|investors|mining|bitum|lignite|coal|opthalm|alum|steel|paint|health|power|advice|chem|patent|patient|facil|plastic|insurance|bank|saving|credit|doctor|medic|surg|pharma|gas|natur|mine|commodity|crude|petroleum|oil|metal|ore|lab|bio|drug|diagnostic'
dfcorps = dfcorps[~dfcorps.SICGroupMinorGroupTitle.str.contains(bad_sics, case=False, na=False)]
dfcorps = dfcorps.dropna(subset=['SICGroupMinorGroupTitle'])

# drop non-focus entities
bad_entities = r'apex \d|mining|liquidating|alternative investment|acquisition|holding|utilities|power|fund|activecare|health|advisor|oxygen|bio|medical|oil|gas|kid doc|mortgage|energy|1847'
dfcorps = dfcorps[~dfcorps.entity_name.str.contains(bad_entities, case=False, na=False)]

# drop bad tickers
dfcorps = dfcorps[(dfcorps.ticker.str.len() > 0) & (dfcorps.ticker.str.len() < 5)]

# save list of calcbench corps to drive
dfcorps.to_csv('data/ciks/calcbench_companies.csv', index=False)

print(f'dfcorps saved successfully!')

dfcorps saved successfully!


In [160]:
# load linkers
dfbcrsp = pd.read_csv('data/ciks/bondcrsp_link.csv')
dftaq = pd.read_csv('data/ciks/taqcrsplink.csv', encoding="ISO-8859-1")

# combine taq and bond links
dftaq = dftaq.rename(columns={'CUSIP': 'EQUITY_CUSIP'})
dftaq = dftaq[['PERMNO', 'EQUITY_CUSIP', 'SYMBOL', 'comnam']]
dfbond_taq = pd.merge(left=dfbcrsp, right=dftaq, how='inner', left_on='PERMNO', right_on='PERMNO')
dfbond_taq = dfbond_taq.drop_duplicates(subset=['CUSIP', 'PERMNO'])

dfcorp_bond_taq = pd.merge(left=dfcorps, right=dfbond_taq, how='inner', left_on='ticker', right_on='SYMBOL')
dfcorp_bond_taq = dfcorp_bond_taq.drop_duplicates(subset=['CUSIP', 'PERMNO'])

dfcusip2 = pd.merge(left=dfcusip, right=dfbond_taq, how='inner', left_on='cusip_id', right_on='CUSIP')
dfcusip2 = dfcusip2.drop_duplicates(subset=['CUSIP', 'PERMNO'])

In [161]:
# clean up columns and save link
colnames = ['cusip_id', 'EQUITY_CUSIP', 'bond_sym_id', 'issuer_nm', 'PERMNO', 'PERMCO', 'SYMBOL']
dfout = dfcusip2[colnames]

dfout.to_csv('data/ciks/bonds_to_equities_link.csv', index=False)

print(f'dfcusip2 saved successfully!')

dfcusip2 saved successfully!


In [165]:
# get tickers
tickers = dfout.SYMBOL.unique()

array(['A', 'AA', 'RTI', 'AMR', 'LCC'], dtype=object)

In [181]:
# pull financials from calcbench
cbh = CBH(save_dir='data/financials')
finished = []
for i in np.arange(3):
    dfs = cbh.fetch_all(company_identifiers=[tickers[i]], start_year=2009,
                       start_period=1, end_year=2019, end_period=4,
                       period_type='quarterly')
    for df in dfs:
        print(f'{i}: {df.name} complete')
        finished.append(df.name)

	save_dir: data/financials
	verbose: True


/home/carbine/miniconda3/envs/fleishco/lib/python3.7/site-packages/calcbench/api_client.py:269: UserWarning: Did not find metrics {'restrictedcashandinvestmentscurrent', 'paymentsofdividendsnoncontrollinginterest', 'secureddebt', 'termloan', 'totalinvestments', 'mediumtermnotes', 'convertibledebt', 'trustpreferredsecurities', 'unsecureddebt', 'longterminvestments', 'mortgagedebt', 'researchanddevelopment', 'subordinateddebt', 'lineofcreditfacilityamountoutstanding', 'paymentsofdividendspreferredstock', 'restructuring'}
  warnings.warn("Did not find metrics {0}".format(missing_metrics))


data saved to data/financials/HD.csv
1002: HD complete


/home/carbine/miniconda3/envs/fleishco/lib/python3.7/site-packages/calcbench/api_client.py:269: UserWarning: Did not find metrics {'paymentsofdividendsnoncontrollinginterest', 'totalinvestments', 'mediumtermnotes', 'trustpreferredsecurities', 'assetimpairment', 'paymentsofdividendscommonstock', 'longterminvestments', 'mortgagedebt', 'researchanddevelopment', 'availableforsalesecurities', 'convertibledebt', 'commonstockdividendspershare', 'paymentsofdividendspreferredstock', 'paymentsofdividends', 'shortterminvestments'}
  warnings.warn("Did not find metrics {0}".format(missing_metrics))


data saved to data/financials/HDS.csv
1004: HDS complete


...

/home/carbine/miniconda3/envs/fleishco/lib/python3.7/site-packages/calcbench/api_client.py:269: UserWarning: Did not find metrics {'paymentsofdividendsnoncontrollinginterest', 'totalinvestments', 'mediumtermnotes', 'convertibledebt', 'trustpreferredsecurities', 'paymentsofdividendscommonstock', 'unsecureddebt', 'mortgagedebt', 'availableforsalesecurities', 'subordinateddebt', 'commonstockdividendspershare', 'paymentsofdividendspreferredstock', 'paymentsofdividends', 'capitalassetsales'}
  warnings.warn("Did not find metrics {0}".format(missing_metrics))


data saved to data/financials/ZBRA.csv
1946: ZBRA complete


/home/carbine/miniconda3/envs/fleishco/lib/python3.7/site-packages/calcbench/api_client.py:269: UserWarning: Did not find metrics {'operatingexpenseexitems', 'restrictedcashandinvestmentscurrent', 'paymentsofdividendsnoncontrollinginterest', 'secureddebt', 'termloan', 'totalinvestments', 'trustpreferredsecurities', 'paymentsofdividendscommonstock', 'longterminvestments', 'mortgagedebt', 'researchanddevelopment', 'restructuring', 'lineofcreditfacilityamountoutstanding', 'currentlongtermdebt', 'paymentsofdividendspreferredstock', 'shortterminvestments'}
  warnings.warn("Did not find metrics {0}".format(missing_metrics))


data saved to data/financials/ZION.csv
1948: ZION complete
data saved to data/financials/QUIK.csv
1949: QUIK complete


/home/carbine/miniconda3/envs/fleishco/lib/python3.7/site-packages/calcbench/api_client.py:269: UserWarning: Did not find metrics {'incometaxespaid', 'secureddebt', 'trustpreferredsecurities', 'paymentsforrepurchaseofcommonstock', 'paymentsofdividendscommonstock', 'interestpaidnet', 'paymentsofdividendspreferredstock', 'paymentsofdividends', 'seniornotes', 'assetimpairment', 'unsecureddebt', 'mortgagedebt', 'stockrepurchasedduringperiodvalue', 'restrictedcashandinvestmentscurrent', 'paymentsofdividendsnoncontrollinginterest', 'termloan', 'totalinvestments', 'mediumtermnotes', 'depreciationandamortizationexpense', 'longterminvestments', 'subordinateddebt', 'stockrepurchasedduringperiodshares', 'convertibledebt', 'commonstockdividendspershare'}
  warnings.warn("Did not find metrics {0}".format(missing_metrics))


(1950,)